# Mixed Integer Linear Programming (MILP) Model for solving airport gate assignment problem

### **Part 1 - Flight-to-Gate Assignment Rules**
### **Part 2 - Model formulation and result**
### **Part 3 - Visualization of gate assignment result using Gantt Chart**

#### *Part 1*
#### Install and import required libraries

In [1]:
!pip install pulp #open source Linear programming (LP) modeler
!pip install openpyxl
!pip install plotly --user

import pandas as pd
import numpy as np
from pulp import *

  Using cached PuLP-2.6.0-py3-none-any.whl (14.2 MB)
  Using cached openpyxl-3.0.10-py2.py3-none-any.whl (242 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


#### Each flight turn can only assign to a compatible parking stand

In [2]:
#import hard-rule such that the type of the aircraft need to match with stand size constraints
aircraft_allowed_stands = pd.read_excel("ac_types_allowed_stands.xlsx", converters={'aircraft_code':str}, sheet_name = 'Stands for Boarding')

In [3]:
#import list of stands available for towing
towing_stands = pd.read_excel("ac_types_allowed_stands.xlsx", converters={'aircraft_code':str}, sheet_name = 'Stands for Towing')
stands_for_towing = towing_stands['allowed_stand'].to_list()

aircraft_allowed_stands = aircraft_allowed_stands.append(towing_stands, ignore_index = True)
aircraft_allowed_stands

,aircraft_code,allowed_stand,code_ref,served_zone
0,388,N5,IATA_code,Green
1,351,N6,IATA_code,Green
2,351,N7,IATA_code,Green
3,351,N9,IATA_code,Green
4,351,N141,IATA_code,Green
...,...,...,...,...
6627,787,S111,AODB_code,towing
6628,777,S111,AODB_code,towing
6629,748,S111,AODB_code,towing
6630,747,S111,AODB_code,towing


#### Due to measures of segregated handling of passengers from different places at the airport, the authority has been handling flights from/to places outside Mainland only in "Orange Zone". For flights from/to Mainland, they will only be allowed to park in "Green Zone"

In [4]:
#Orange Zone: arrival_aircraft_allowed_stands
oz_arrival_aircraft_allowed_stands = aircraft_allowed_stands.loc[aircraft_allowed_stands['served_zone'].isin(["Orange","Green_Orange"])]
oz_arrival_stand_list = list(oz_arrival_aircraft_allowed_stands['allowed_stand'].unique())

#Orange Zone: departure_aircraft_allowed_stands, excl. S1-S4 where <served_zone> = "Green_Orange"
oz_departure_aircraft_allowed_stands = aircraft_allowed_stands.loc[aircraft_allowed_stands['served_zone'] == "Orange"]
oz_departure_stand_list = list(oz_departure_aircraft_allowed_stands['allowed_stand'].unique())

#Green Zone: arrival_aircraft_allowed_stands
#N5-N9; (Remote) N141-N145
gz_arrival_aircraft_allowed_stands = aircraft_allowed_stands.loc[aircraft_allowed_stands['served_zone'] == "Green"]
gz_arrival_stand_list = list(gz_arrival_aircraft_allowed_stands['allowed_stand'].unique())

#Green Zone: departure_aircraft_allowed_stands
#N5-N9; (Remote) N141-N145; S1-S4
gz_departure_aircraft_allowed_stands = aircraft_allowed_stands.loc[aircraft_allowed_stands['served_zone'].isin(["Green","Green_Orange"])]
gz_departure_stand_list = list(gz_departure_aircraft_allowed_stands['allowed_stand'].unique())

#### Import flight schedule raw data

In [5]:
#import flight schedule data
turns = pd.read_excel("20Nov (Linked Flight).xlsx", converters={'Actyp':str})

#Add unique id for each flight record
turns.insert(0, 'turn_no', range(1, 1 + len(turns)))

In [6]:
#Identify Zone Group at the times of arrival and departure
turns['arrival_zone'] = turns['Last Country'].apply(lambda x: 'Green' if x=='CN' else 'Orange')
turns['departure_zone'] = turns['Next Country'].apply(lambda x: 'Green' if x=='CN' else 'Orange')
turns['from_to_same_zone'] = np.where((turns['arrival_zone'] == turns['departure_zone']), 1, 0)
turns.head()

,turn_no,OpeName,Airl.Desig,A Fltno,Arr Time,D Fltno,Dep Time,Actyp,Seats,Serv.type,...,Next,Dest,Next City Served,Next Country,Arrival Zone,Departure Zone,Zone Pair,arrival_zone,departure_zone,from_to_same_zone
0,1,Sichuan Airlines,3U,3959,2022-11-20 17:00:00,3960,2022-11-20 18:30:00,330,305,J,...,CTU,CTU,Chengdu,CN,Green,Green,G-G,Green,Green,1
1,2,Fly Gangwon,4V,241,2022-11-20 00:50:00,242,2022-11-20 02:05:00,738,186,J,...,YNY,YNY,Yangyang,KR,Orange,Orange,O-O,Orange,Orange,1
2,3,Cebu Pacific,5J,272,2022-11-20 08:10:00,273,2022-11-20 09:25:00,333,436,J,...,MNL,MNL,Manila,PH,Orange,Orange,O-O,Orange,Orange,1
3,4,Cebu Pacific,5J,110,2022-11-20 09:45:00,111,2022-11-20 11:00:00,339,459,J,...,MNL,MNL,Manila,PH,Orange,Orange,O-O,Orange,Orange,1
4,5,Cebu Pacific,5J,112,2022-11-20 18:00:00,113,2022-11-20 19:15:00,32Q,236,J,...,MNL,MNL,Manila,PH,Orange,Orange,O-O,Orange,Orange,1


#### Towing protocol will be applied for the following cases:
#### 1) Over 6 hours of ground time will be towed away from the stand
#### 2) The pair of flights do not correspond to the same zone (i.e. Green-Orange or Orange-Green)

In [7]:
towing_threshold_hours = 6

# #Towing plane indicator
turns['ground_time'] = (turns['Dep Time'] - turns['Arr Time']).astype('timedelta64[h]')
turns['towing_required'] = np.where((turns['ground_time'] > towing_threshold_hours) | (turns['from_to_same_zone'] == 0), 1, 0)

# Filter on eligible turns for towing and duplicate
eligible_for_tow = turns[turns['towing_required'] ==1]
eligible_for_tow = eligible_for_tow.append(eligible_for_tow, ignore_index=True)

# Add back to original
turns = turns.append(eligible_for_tow, ignore_index=True)

# Turn_part
turns['turn_part'] = turns.groupby('turn_no')['turn_no'].rank(method='first')

# Turn_part = 0 if no tows
cnt_parts = turns.groupby("turn_no")["turn_no"].transform('count')
turns.loc[cnt_parts==1, "turn_part"] = 0

# Sort
turns.sort_values(['turn_no', 'turn_part'], inplace=True)

#### For towing plane, add additional 90 mins on top of ATD to cater for boarding, departing the plane and the physical process of towing. Besides, the aircraft needs to tow back to the compatible stand to prepare for embarking 1 hour prior to its STD

In [8]:
# Part 1
turns.loc[turns.turn_part==1, "Dep Time"] = turns['Arr Time'] + pd.offsets.Minute(90)
# Part 2
turns.loc[turns.turn_part==2, "Arr Time"] = turns['Arr Time'] + pd.offsets.Minute(90)
turns.loc[turns.turn_part==2, "Dep Time"] = turns['Dep Time'] - pd.offsets.Minute(60)

# Part 3
turns.loc[turns.turn_part==3, "Arr Time"] = turns['Dep Time'] - pd.offsets.Hour(1)

In [9]:
# Create new-id and reset columns
turns.reset_index(drop=True, inplace=True)
turns.rename(columns={'turn_no': 'original_turn'}, inplace=True)
turns['turn_no'] = turns.index

#### Exclude turns whose zone pair is "Orange-Orange" (i.e. Origin-Destination is non-Mainland China region)

In [10]:
turns = turns.loc[turns['Zone Pair']!="O-O"].reset_index(drop=True)
turns['turn_no'] = turns.index

#### On grounds of airfield safety operation, at least 25 minutes interval (“time gap”) in-between slot of each aircraft occupied in a parking stand. For each parking stand, therefore, no flight-to-gate assignment exercise shall be made 12.5 minutes before the scheduled time of arrival (STA) of arriving flights until 12.5 minutes after the scheduled time of departure (STD) of departing flights

In [11]:
#Set 25 mins as a buffer time of stand allocation slot in-between each turn in a parking stand
#From 12.5 minutes before the scheduled time of arrival (STA) until 12.5 minutes after the scheduled time of departure (STD)
min_buffer_threshold = 750

turns['occupied_stand_from'] = turns['Arr Time']
turns['occupied_stand_to'] = turns['Dep Time']

turns.loc[turns.turn_part!=2, "occupied_stand_from"] = turns['occupied_stand_from'] - pd.offsets.Second(min_buffer_threshold)
turns.loc[turns.turn_part!=2, "occupied_stand_to"] = turns['occupied_stand_to'] + pd.offsets.Second(min_buffer_threshold)

In [12]:
stand_list = pd.Series(aircraft_allowed_stands['allowed_stand']).unique()
turn_list = turns['turn_no'].to_numpy()

#### Create a dictionary to store values of compatible stand values for each turn

In [13]:
compatible_stands = {}
for idx, row in turns.iterrows():
    gates_lst = aircraft_allowed_stands[aircraft_allowed_stands.aircraft_code == row.Actyp].allowed_stand.to_numpy()
    for g in gates_lst:
        if (row['arrival_zone'] == 'Green') and (row['turn_part'] in (0,1)):
            gates_lst = [ele for ele in gates_lst if ele in gz_arrival_stand_list]
        
        if (row['arrival_zone'] == 'Orange') and (row['turn_part'] in (0,1)):
            gates_lst = [ele for ele in gates_lst if ele in oz_arrival_stand_list]
        
        if (row['departure_zone'] == 'Green') and (row['turn_part'] ==3):
            gates_lst = [ele for ele in gates_lst if ele in gz_departure_stand_list]
            
        if (row['departure_zone'] == 'Orange') and (row['turn_part'] ==3):
            gates_lst = [ele for ele in gates_lst if ele in oz_departure_stand_list]
            
        if row['turn_part'] == 2:
            gates_lst = [ele for ele in gates_lst if ele in stands_for_towing]
            
        compatible_stands[row.turn_no] = np.array(gates_lst)

# print("Compatible stands for each turn:")
# for k, v in compatible_stands.items():
#     print(k, v)

#### Parking stands can serve more than one turn in a day if they don't overlap.
#### Set a 5-minute interval as index of time-series ranging from STA of first flight and STD of the last flight. Columns are binary variables to indicate whether the turn is at the airport

In [14]:
# Using discrete time-buckets
min_bucket=5

# Create time-series between arrival of first plane and departure of last
time_series = pd.Series(True, index= pd.date_range(
        start=turns['occupied_stand_from'].min(),
        end=turns['occupied_stand_to'].max(),
        freq=pd.offsets.Minute(min_bucket)))
    
# Truncate full time-series to [STA, STD]
def trunc_ts(series):
    return time_series.truncate(series['occupied_stand_from'], series['occupied_stand_to'])
    
stand_occupacy_df = turns.apply(trunc_ts, axis=1).T
    
# Convert columns from index to turn_no
stand_occupacy_df.columns = turns['turn_no'].values

# Cast to integer
stand_occupacy_df = stand_occupacy_df.fillna(0).astype(int)
stand_occupacy_df.head()

,0,1,2,3,4,5,6,7,8,9,...,128,129,130,131,132,133,134,135,136,137
2022-11-19 11:47:30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-11-19 11:52:30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-11-19 11:57:30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-11-19 12:02:30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-11-19 12:07:30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Only care about overlaps
# If the stand only has one turn then don't need constraint that it must have one turn ...
stand_occupacy_df['tot'] = stand_occupacy_df.sum(axis=1)
stand_occupacy_df = stand_occupacy_df[stand_occupacy_df.tot > 1]
stand_occupacy_df.drop(['tot'], axis=1, inplace=True)

#### *Part 2*
#### Formulate Model and find the values of variables (i,j) for each turn such that all constraints are satisfied and the objective function is minimized.

In [16]:
# 0. Initialise model
prob = LpProblem("Flight_Stand_Allocation", LpMinimize)  # minimize cost

# 1. Set the objective function (ignore 0 for now)
prob += 0

# 2. Define Variable: x[i,j] = (0,1)
# Binary = turn_i allocated to gate_j
x = {}
for t in turn_list:
    for g in compatible_stands[t]:
        x[t, g] = LpVariable("t%i_g%s" % (t, g), 0, 1, LpBinary)

# 3. Constraints
# i. Each turn must be assigned to one compatible stand
for t in turn_list:
    prob += lpSum(x[t, g] for g in stand_list if (t, g) in x) == 1

In [17]:
# ii. Stands cannot serve more than one turn per time_bucket
for idx, row in stand_occupacy_df.iterrows():
    # Get all the turns for time-bucket
    turns_in_time_bucket = set(dict(row[row==1]).keys())

    for g in stand_list:
        # Constraints may be blank
        cons = [x[t, g] for t in turns_in_time_bucket if (t, g) in x]
        if len(cons) > 1:
            constraint_for_time_bucket = lpSum(cons) <= 1
            prob += constraint_for_time_bucket

In [18]:
# iii. Tow-constraint
# Create dictionary for tows of turn_no ({<turn_no>: towing(<turn_no))...})
# Create a tow-variable (w) (similar to the allocation-variable: x). Tow if w = 1, otherwise 0.
w = {} 

# List of turns for tows
tow_turns = turns[turns['turn_part']>0]['turn_no'].to_numpy()

# Each towing turn must be assigned to one towing stand
for tow_turn in tow_turns:
    w[tow_turn] = LpVariable("towing(%s)" % (tow_turn), 0, 1, LpBinary)
    
# Create towing dictionary to map original turn to new turns (to group them)
# For example, original_turn id #20 would split into turn_no #19,#20,#21
tow_dic = {k: g["turn_no"].tolist() for k,g in turns[turns['turn_part']>0].groupby("original_turn")}

##### For tow-out: prob += x[tow_turns[0], stand] - x[tow_turns[1], stand] <= w[tow_turns[0]]
##### For tow-in: prob += x[tow_turns[1], stand] - x[tow_turns[2], stand] <= w[tow_turns[1]]

In [19]:
for k, v in tow_dic.items():
    #print(k, v)
    for stand in stand_list:
        if ((v[0], stand) in x) and ((v[1], stand) in x):
            tow_out = x[v[0], stand] - x[v[1], stand] <= w[v[0]]
            prob += tow_out
        if ((v[1], stand) in x) and ((v[2], stand) in x):
            tow_in =  x[v[1], stand] - x[v[2], stand] <= w[v[1]]
            prob += tow_in

In [20]:
# Add towing objective with positive cost
pos_cost_coefficient = 100
tow_objective = lpSum(pos_cost_coefficient*w[t] for t in w)

prob += tow_objective

/opt/conda/lib/python3.7/site-packages/pulp/pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


#### Assignment preference: 1st priority: Frontal Stand; (2) Remote Stand (N141-N145)

In [21]:
# Add all turns under remote stands with a negative cost coefficient
remote_stands = ['N141','N142','N143','N144','N145']

neg_cost_coefficient = -5
prob += lpSum(neg_cost_coefficient*x[t, g] for t, g in x if g not in remote_stands)

#### *Part 3*
#### Run the Solver Algorithm to find optimal solution. The default solver used by PuLP is the COIN-OR Branch and Cut Solver (CBC).

In [22]:
# Solve
prob.solve()

# Report
print("Status: ", LpStatus[prob.status])
print("Minimised Cost: ", value(prob.objective))

for alloc in x:
    if x[alloc].varValue:
        print("Turn %i assigned to gate %s" % (alloc[0], alloc[-1]))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/9b89701c19eb4d0e8b9e73ca4c33868f-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/9b89701c19eb4d0e8b9e73ca4c33868f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 51726 COLUMNS
At line 542710 RHS
At line 594432 BOUNDS
At line 598205 ENDATA
Problem MODEL has 51721 rows, 3772 columns and 480032 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.76 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       1.08   (Wallclock seconds):       1.14

Status:  Infeasible
Minimised Cost:  -610.0
Turn 0 assigned to gate N144
Turn 1 assigned to gate N6
Turn 2 assigned to gate N142
Turn 3 assigned to gate N9
Turn 4 assigned to gate N9
Turn 5 assigned to gate N141
Turn 6 assig

In [23]:
## Save the solver result into a new column "assigned_stand" in df
results = {}

for alloc in x:
    if x[alloc].varValue:
        dicts = dict({alloc[0]: alloc[-1]})
        results.update(dicts)
    
turns['assigned_stand'] = turns['turn_no'].map(results)
turns.head()

,original_turn,OpeName,Airl.Desig,A Fltno,Arr Time,D Fltno,Dep Time,Actyp,Seats,Serv.type,...,arrival_zone,departure_zone,from_to_same_zone,ground_time,towing_required,turn_part,turn_no,occupied_stand_from,occupied_stand_to,assigned_stand
0,1,Sichuan Airlines,3U,3959,2022-11-20 17:00:00,3960,2022-11-20 18:30:00,330,305,J,...,Green,Green,1,1.0,0,0.0,0,2022-11-20 16:47:30,2022-11-20 18:42:30,N144
1,8,Spring Airlines,9C,6169,2022-11-20 16:30:00,6170,2022-11-20 17:30:00,320,180,J,...,Green,Green,1,1.0,0,0.0,1,2022-11-20 16:17:30,2022-11-20 17:42:30,N6
2,9,Spring Airlines,9C,6113,2022-11-20 17:55:00,6114,2022-11-20 18:55:00,320,180,J,...,Green,Green,1,1.0,0,0.0,2,2022-11-20 17:42:30,2022-11-20 19:07:30,N142
3,10,Spring Airlines,9C,8587,2022-11-20 18:25:00,8588,2022-11-20 19:25:00,320,180,J,...,Green,Green,1,1.0,0,0.0,3,2022-11-20 18:12:30,2022-11-20 19:37:30,N9
4,11,Spring Airlines,9C,6175,2022-11-20 20:25:00,6176,2022-11-20 21:25:00,320,180,J,...,Green,Green,1,1.0,0,0.0,4,2022-11-20 20:12:30,2022-11-20 21:37:30,N9


In [24]:
turns.to_excel('output.xlsx', index=False)

In [25]:
available_stand_list = pd.read_excel("available_stand_list.xlsx")

In [29]:
import plotly #open-source graphing libraries
import plotly.express as px

fig = px.timeline(turns, x_start="Arr Time", x_end="Dep Time", y="assigned_stand", color="assigned_stand",
                  hover_name='turn_no',
                  width=1400, height=2500)

# Set assigned stand order in y-axis
fig.update_yaxes(categoryarray= available_stand_list['stands'].to_list())

# Add range slider
fig.update_layout(bargap=0.5, bargroupgap=0.1, title='Departing Pax Flight Assignment Simulation Result',
      xaxis=dict(
          rangeselector=dict(
              buttons=list([
                  dict(step="all")
              ])
          ),
          rangeslider=dict(
              visible=True
          ),
          type="date"
      )
  )                  

fig.show()
fig.write_html('index.html', include_plotlyjs='cdn')

KeyError: (nan, '', '', '')